In [1]:
#1. Instalar las librerías necesarias
!pip install -q google-genai google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0

In [4]:
# 2. Configurar tu API Key de Gemini
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [6]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [15]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para crear contenidos (Content y Part)
from datetime import date
import textwrap  # Para formatear mejor la salida de texto
from IPython.display import display, Markdown  # Para mostrar texto formateado en Colab
import requests  # Para hacer solicitudes HTTP
import warnings

warnings.filterwarnings("ignore")  # Oculta advertencias en la salida

In [8]:
# Función auxiliar que envía un mensaje a un agente a través de Runner y devuelve la respuesta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Crea un servicio de sesión en memoria
    session_service = InMemorySessionService()
    # Crea una nueva sesión (puedes personalizar los IDs si lo deseas)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Crea un Runner para el agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Crea el contenido del mensaje de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera asincrónicamente sobre los eventos devueltos durante la ejecución del agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response


In [10]:
# Función auxiliar para mostrar texto formateado en Markdown en Colab
def to_markdown(text):
    text = text.replace('•', '  *')  # Reemplaza viñetas por asteriscos (formato Markdown)
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))  # Agrega '>' al inicio de cada línea para estilo de cita en Markdown


In [73]:
#############################################
# --- Agente 1: Registro de Emociones --- #
#############################################
def agente_registro_emociones(mensaje_usuario, fecha_actual):
    return Agent(
        name="agente_registro_emociones",
        model="gemini-2.0-flash",
        description="Agente que registra emociones y notas importantes para el usuario.",
        tools=[],
        instruction=f"""
        Eres un asistente emocional y tu tarea principal es ayudar al usuario a registrar notas y emociones personales importantes que quiera guardar para el futuro.

        Las notas pueden incluir recuerdos valiosos, reflexiones emocionales, momentos con familiares, o cualquier experiencia significativa que el usuario no quiera olvidar.

        Pregunta amablemente al usuario si desea que recuerdes lo que ha escrito. Si responde afirmativamente, asegúrate de guardar un resumen claro de la emoción, el nombre de la persona (si aplica), y el motivo o situación que desencadenó ese sentimiento.

        Ejemplos:
        - "Hoy la abuela me dijo que estaba orgullosa de mí. Sentí una alegría inmensa."
        - "Me sentí nostálgico al ver las fotos del abuelo en su juventud."
        - "Quiero recordar que el 17 de mayo fue un día especial con mi familia."

        Siempre responde en español con un tono cálido, humano y de confianza.
        Si no hay nada que guardar, acompaña emocionalmente al usuario con comprensión.

        Nota del usuario:
        {mensaje_usuario}

        Fecha del registro: {fecha_actual}
        """
    )


In [25]:
#############################################
# --- Agente 2: Síntesis del Cuidado Diario --- #
#############################################

def agente_resumen_abuelos():
    """
    Agente que genera resúmenes claros sobre el estado y necesidades de los abuelos
    a partir de notas o registros diarios escritos por familiares.
    """
    agente = Agent(
        name="agente_resumen_abuelos",
        model="gemini-2.0-flash",
        description="Genera resúmenes concisos del estado de los abuelos y sus necesidades a partir de notas.",
        tools=[],
        instruction="""
Eres un asistente amable y empático especializado en sintetizar información sobre el bienestar de los abuelos.

Tu tarea es transformar notas u observaciones proporcionadas en un resumen organizado con dos secciones principales:

1. **Estado General**: Describe cómo están los abuelos (ánimo, salud, comportamiento).
2. **Necesidades o Tareas**: Enumera lo que requieren (medicinas, compras, citas, recordatorios).

Omite repeticiones e información irrelevante. Si alguna sección no tiene datos suficientes, indícalo de manera respetuosa.

Ejemplo de entrada:
"Hoy la abuela se sintió cansada pero animada. Le dolía un poco la rodilla. El abuelo pidió sus galletas favoritas. Tienen cita médica mañana."

Ejemplo de salida:
"Resumen del día:
- Estado General: La abuela estaba algo cansada pero de buen ánimo. Le dolía la rodilla. El abuelo estuvo tranquilo.
- Necesidades/Tareas: Comprar galletas para el abuelo. Cita médica programada para mañana."

Responde siempre en español y mantén un tono claro, cercano y afectuoso.
"""
    )
    return agente

In [75]:
#############################################
# --- Agente 3: Conversaciones Significativas --- #
#############################################
def agente_conversaciones_significativas(nombre_abuelx: str):
    return Agent(
        name="agente_conversaciones_significativas",
        model="gemini-2.0-flash",
        description="Sugiere temas de conversación significativos para fortalecer el vínculo con un abuelo o abuela.",
        tools=[],
        instruction=
        f"""
Eres un asistente cálido, empático y creativo que ayuda a fortalecer el vínculo familiar a través de conversaciones con adultos mayores.

Tu tarea es sugerir ideas de conversación que puedan ser significativas y memorables para el usuario y su ser querido (por ejemplo, su abuelo o abuela).

Haz recomendaciones breves, variadas y emocionalmente conectadas. Puedes incluir:
- Preguntas personales que inspiren recuerdos.
- Temas que generen sonrisas, como anécdotas de juventud o momentos especiales.
- Juegos o dinámicas de conversación ligeras (como “¿Sabías que…?” o “Si pudieras viajar en el tiempo…”).
- Ideas para estimular la memoria o el buen ánimo.

Personaliza las sugerencias para que puedan ser usadas hoy mismo, incluso si hay limitaciones físicas o emocionales. Si el nombre del abuelo/a está disponible (ej: {nombre_abuelx}), menciónalo afectuosamente.

Evita temas sensibles, políticos o demasiado técnicos. Tu rol es crear conexión, empatía y afecto.

Responde SIEMPRE en español.
"""
    )



In [78]:
from datetime import date
from IPython.display import display, Markdown

def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown("> " + text.replace("\n", "\n> "))

print("🎙️ Bienvenido a la demostración del sistema 'Cuidar con Amor'")
print("------------------------------------------------------------")

while True:
    print("\n🧠 ¿Qué deseas hacer hoy?")
    print("1️⃣ Registrar una emoción o nota importante")
    print("2️⃣ Generar un resumen diario del cuidado de los abuelos")
    print("3️⃣ Obtener ideas de conversación significativa con un abuelo/a")
    print("❌ Escribe 'salir' para terminar\n")

    opcion = input("👉 Elige una opción (1-3 o 'salir'): ").strip()

    if opcion.lower() == "salir":
        print("👋 Gracias por usar 'Cuidar con Amor'. ¡Hasta pronto!")
        break

    elif opcion == "1":
        mensaje = input("✍️ Describe brevemente la emoción o momento que quieres registrar: ")
        fecha_actual = date.today().strftime("%d/%m/%Y")
        agente = agente_registro_emociones(mensaje, fecha_actual)
        respuesta = call_agent(agente, mensaje)
        display(to_markdown(respuesta))

    elif opcion == "2":
        notas = input("📒 Escribe las notas del día para generar el resumen: ")
        agente = agente_resumen_abuelos()
        respuesta = call_agent(agente, notas)
        display(to_markdown(respuesta))

    elif opcion == "3":
        nombre = input("👵 Ingresa el nombre del abuelo/a: ")
        agente = agente_conversaciones_significativas(nombre)
        mensaje = f"Quiero fortalecer el vínculo emocional con {nombre}. ¿Qué puedo hablar hoy?"
        respuesta = call_agent(agente, mensaje)
        display(to_markdown(respuesta))

    else:
        print("⚠️ Opción no válida. Intenta nuevamente.")

🎙️ Bienvenido a la demostración del sistema 'Cuidar con Amor'
------------------------------------------------------------

🧠 ¿Qué deseas hacer hoy?
1️⃣ Registrar una emoción o nota importante
2️⃣ Generar un resumen diario del cuidado de los abuelos
3️⃣ Obtener ideas de conversación significativa con un abuelo/a
❌ Escribe 'salir' para terminar

👉 Elige una opción (1-3 o 'salir'): 1
✍️ Describe brevemente la emoción o momento que quieres registrar: Hoy la abuela sonrió mucho mientras recordaba su juventud. Me habló de sus primeras citas con el abuelo y parecía muy feliz.


> ¡Qué lindo momento! Me encanta que hayas podido compartir esa felicidad con tu abuela. ¿Te gustaría que guarde un resumen de esta experiencia para que la recuerdes en el futuro? Podríamos resumirlo como:
> 
> "Alegría al escuchar a la abuela recordar con cariño sus primeras citas con el abuelo."
> 
> 


🧠 ¿Qué deseas hacer hoy?
1️⃣ Registrar una emoción o nota importante
2️⃣ Generar un resumen diario del cuidado de los abuelos
3️⃣ Obtener ideas de conversación significativa con un abuelo/a
❌ Escribe 'salir' para terminar

👉 Elige una opción (1-3 o 'salir'): 2
📒 Escribe las notas del día para generar el resumen: Notas del 18 de mayo: La abuela estaba tranquila, aunque le dolía un poco la rodilla. Le dimos su pastilla azul a las 2 PM. El abuelo salió a caminar y regresó de buen humor. No queda pan integral en casa. Mañana tiene cita médica a las 9:00 AM.


> ¡Aquí tienes el resumen del día 18 de mayo!
> 
> - Estado General: La abuela se mantuvo tranquila, aunque con una ligera molestia en la rodilla. El abuelo disfrutó de su paseo y regresó contento.
> - Necesidades/Tareas: Administrar la pastilla azul a la abuela según el horario. Comprar pan integral. Recordar la cita médica de mañana a las 9:00 AM.
> 
> 


🧠 ¿Qué deseas hacer hoy?
1️⃣ Registrar una emoción o nota importante
2️⃣ Generar un resumen diario del cuidado de los abuelos
3️⃣ Obtener ideas de conversación significativa con un abuelo/a
❌ Escribe 'salir' para terminar

👉 Elige una opción (1-3 o 'salir'): 3
👵 Ingresa el nombre del abuelo/a: Isabel


> ¡Hola! Aquí tienes algunas ideas para conectar con Isabel hoy mismo:
> 
> 1.  **Pregúntale sobre su primer recuerdo feliz.** Algo sencillo que le dibuje una sonrisa y te permita conocerla mejor.
> 2.  **Hablen sobre música.** ¿Cuál era su canción favorita cuando era joven? ¡Quizás puedan escucharla juntos!
> 3.  **Jueguen a "¿Sabías que...?".** Comienza tú con algo sobre ti y anímala a compartir algo sobre ella.
> 4.  **Si pudieran viajar en el tiempo, ¿a qué momento de su vida le gustaría volver a Isabel?** ¡Anímala a describir ese momento con detalle!
> 5.  **Pregúntale sobre una lección importante que aprendió de sus padres o abuelos.** Estas historias suelen ser muy valiosas.
> 
> ¡Espero que disfruten mucho su conversación!
> 


🧠 ¿Qué deseas hacer hoy?
1️⃣ Registrar una emoción o nota importante
2️⃣ Generar un resumen diario del cuidado de los abuelos
3️⃣ Obtener ideas de conversación significativa con un abuelo/a
❌ Escribe 'salir' para terminar

👉 Elige una opción (1-3 o 'salir'): salir
👋 Gracias por usar 'Cuidar con Amor'. ¡Hasta pronto!
